### Verificação dos dados na camada Silver

In [1]:
# Bibliotecas
import pandas as pd
from sqlalchemy import create_engine, inspect, text


In [ ]:
# Conexão com o banco de dados
db_path_silver = "../database/silver/01_silver.db"
engine = create_engine(f"sqlite:///{db_path_silver}")

In [ ]:
# 1) Total de vendas ConsigCar por tipo de produto
query1 = """
SELECT tipo_produto, COUNT(*) AS quantidade_vendas, SUM(p.valor_total) AS valor_total_vendas
FROM fato_vendas_consigcar f
LEFT JOIN dim_parcelamento p ON f.id_parcelamento = p.id_parcelamento
GROUP BY tipo_produto
ORDER BY valor_total_vendas DESC
LIMIT 8;
"""

df1 = pd.read_sql(query1, engine)
print(df1)

In [ ]:
# X)Vendas mensais por vendedor com nome do mês
query_2 = """
SELECT
  v.id_vendedor,
  v.nome_vendedor,
  t.mes,
  t.nome_mes,
  SUM(f.quantidade_vendas)    AS total_vendas,
  SUM(CAST(f.valor_total AS FLOAT)) AS total_valor_vendido
FROM fato_vendas_mensais_consigcar f
JOIN dim_vendedor v ON f.id_vendedor = v.id_vendedor
JOIN dim_tempo t     ON f.id_data    = t.id_data
GROUP BY v.id_vendedor, v.nome_vendedor, t.mes, t.nome_mes
ORDER BY t.ano DESC, t.mes DESC
LIMIT 8;
"""

df2 = pd.read_sql(query_2, engine)
print(df2)

In [17]:
# 3) Parcelamentos ativos e valores totais
query3 = """
SELECT id_parcelamento, num_parcelas, valor_parcela, valor_total, data_primeira_parcela, data_ultima_parcela
FROM dim_parcelamento
ORDER BY data_primeira_parcela DESC
LIMIT 8;
"""
df3 = pd.read_sql(query3, engine)
df3

,id_parcelamento,num_parcelas,valor_parcela,valor_total,data_primeira_parcela,data_ultima_parcela
0,105,6,611.45,3668.70,20310125,20310624
1,36,12,1236.66,14839.92,20301124,20311020
2,195,12,1225.62,14707.44,20300425,20310321
3,196,6,627.75,3766.50,20300425,20300922
4,197,18,1841.90,33154.20,20300425,20310917
5,198,18,1839.90,33118.20,20300425,20310917
6,199,18,1830.50,32949.00,20300425,20310917
7,200,18,1848.71,33276.78,20300425,20310917


In [ ]:
# X) Despesas por categoria e mês
query4 = """
SELECT ano, mes, SUM(valor) AS total_despesas
FROM fato_despesas d
JOIN dim_tempo t ON id_data
GROUP BY ano, mes
ORDER BY ano DESC, mes DESC
LIMIT 8;
"""

df4 = pd.read_sql(query4, engine)
print(df4)

OperationalError: (sqlite3.OperationalError) ambiguous column name: id_data
[SQL: 
SELECT ano, mes, SUM(d.valor) AS total_despesas
FROM fato_despesas d
JOIN dim_tempo t ON id_data
GROUP BY ano, mes
ORDER BY ano DESC, mes DESC
LIMIT 8;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [21]:
# 5) Select simples
query5 = """
SELECT * FROM fato_vendas_consigcar
"""
df5 = pd.read_sql(query5, engine)
df5.head(10)

,id_venda_consigcar,id_cliente,tipo_produto,id_parcelamento,id_vendedor,id_data
0,1,Fernanda Gomes Ibrahim Campos,Completa,1,Thiago,20241111.0
1,2,Patrícia Zanetti Jardim Campos Souza,Completa,2,Thiago,NaN
2,3,Eduardo Yamada Dias Vieira Nogueira,Completa,3,Thiago,NaN
3,4,Thiago Lopes Xavier Queiroz,Completa,4,Thiago,NaN
4,5,Vanessa Yamada Farias Krause Teixeira,Completa,5,Thiago,NaN
5,6,Natália Oliveira Vieira Lopes Nogueira Costa,Completa,6,Thiago,20240111.0
6,7,Patrícia Machado Teixeira Zanetti Ibrahim Queiroz,Completa,7,Fernanda,20240110.0
7,8,Kleber Nogueira Costa Teixeira Silva Esteves,Completa,8,Thiago,20240710.0
8,9,Helena Oliveira Nogueira Esteves Zanetti,Completa,9,Fernanda,20240611.0
9,10,Fernanda Rodrigues Costa Uchoa Almeida,Completa,10,Fernanda,20240911.0
